In [11]:
import dask
dask.config.set({"dataframe.query-planning": False})

import itertools
import scanpy as sc
import numpy as np
import pandas as pd
import sys
import argparse
import json
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import _utils
import preprocessing
import readwrite
cfg = readwrite.config()

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
counts = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/seurat_to_h5/matched_combo_standard_lung_specific/RNA_counts.h5'
metadata = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/results/seurat_to_h5/matched_combo_standard_lung_specific/metadata.parquet'
genes = pd.read_csv('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/markers/Xenium_hLung_v1_metadata.csv')['Gene'].tolist()

# params from pipeline config
min_counts = 10
min_features = 5
max_counts = float("inf")
max_features = float("inf")
min_cells = 5


adata = sc.read_10x_h5(counts)
adata.obs = pd.read_parquet(metadata)
adata.obs.loc[adata.obs['Level2.1'].str.contains("malignant") ,'Level2.1']='malignant cell'

if len(genes):
    print("Subsetting")

    genes_found = [
        g
        for g in adata.var_names
        if (g in genes) or (g.replace(".", "-") in genes)  # possible seurat renaming
    ]

    print(f"Found {len(genes_found)} out of {len(genes)} genes.")

    # read raw counts to reapply QC
    adata = adata[:, genes_found].copy()

    # reapply QC to subset of genes
    preprocessing.preprocess(
        adata,
        min_counts=min_counts,
        min_genes=min_features,
        max_counts=max_counts,
        max_genes=max_features,
        min_cells=min_cells,
        save_raw=False,
    )

Subsetting
Found 284 out of 289 genes.
Removed 8283  cells...
Removed 1  genes...
GPU not available. Switching to CPU backend...


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [25]:
adata.obs['Level2.1'].value_counts()

Level2.1
plasma cell                        5492
macrophage                         4241
CD4-positive, alpha-beta T cell    4233
CD8-positive, alpha-beta T cell    3596
fibroblast of lung                 2177
malignant cell                     2177
dendritic cell                     1909
B cell                             1534
smooth muscle cell                 1529
monocyte                           1394
endothelial cell                    990
T cell                              611
epithelial cell of lung             589
natural killer cell                 587
mast cell                           511
cycling lymphocyte                  381
pericyte                            314
granulocyte                          52
Name: count, dtype: int64